In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
import geopandas as gpd

FIRST READ DATA

In [2]:
df_counties_original = pd.read_csv("D:\\UIUC\\COVID_19\\covid-19-data\\us-counties.csv", dtype={"date": np.string_, 
                                                             "county": np.string_,
                                                             "state": np.string_,
                                                             "fips": np.string_,
                                                            "cases": np.int32,
                                                            "deaths": np.int32})
df_counties_original['fips'] = df_counties_original['fips'].replace({np.nan: '00000'})
df_counties_original.sort_values("fips", ascending=True)

,date,county,state,fips,cases,deaths
13531,2020-03-26,Unknown,Ohio,00000,1,0
29576,2020-04-03,Unknown,Minnesota,00000,0,3
29699,2020-04-03,Kansas City,Missouri,00000,153,1
29735,2020-04-03,Unknown,Missouri,00000,3,0
29828,2020-04-03,Unknown,New Jersey,00000,4805,4
29850,2020-04-03,Unknown,New Mexico,00000,2,0
29880,2020-04-03,New York City,New York,00000,57160,1867
29904,2020-04-03,Unknown,New York,00000,37,608
30022,2020-04-03,Unknown,Northern Mariana Islands,00000,8,1
29523,2020-04-03,Unknown,Michigan,00000,215,1


Generate GeoJSON

In [3]:
pivot_counties = pd.pivot_table(df_counties_original , index=['state','county','fips'],
                       columns=['date'])
pivot_counties

cases                                   \
date                        2020-01-21 2020-01-22 2020-01-23 2020-01-24   
state     county      fips                                                
Alabama   Autauga     01001        NaN        NaN        NaN        NaN   
          Baldwin     01003        NaN        NaN        NaN        NaN   
          Barbour     01005        NaN        NaN        NaN        NaN   
          Bibb        01007        NaN        NaN        NaN        NaN   
          Blount      01009        NaN        NaN        NaN        NaN   
          Bullock     01011        NaN        NaN        NaN        NaN   
          Butler      01013        NaN        NaN        NaN        NaN   
          Calhoun     01015        NaN        NaN        NaN        NaN   
          Chambers    01017        NaN        NaN        NaN        NaN   
          Cherokee    01019        NaN        NaN        NaN        NaN   
          Chilton     01021        NaN        NaN        NaN        NaN   
          Choctaw     01023        NaN        NaN        NaN        NaN   
          Clarke      01025        NaN        NaN        NaN        NaN   
          Clay        01027        NaN        NaN        NaN        NaN   
          Cleburne    01029        NaN        NaN        NaN        NaN   
          Coffee      01031        NaN        NaN        NaN        NaN   
          Colbert     01033        NaN        NaN        NaN        NaN   
          Conecuh     01035        NaN        NaN        NaN        NaN   
          Coosa       01037        NaN        NaN        NaN        NaN   
          Covington   01039        NaN        NaN        NaN        NaN   
          Crenshaw    01041        NaN        NaN        NaN        NaN   
          Cullman     01043        NaN        NaN        NaN        NaN   
          Dale        01045        NaN        NaN        NaN        NaN   
          Dallas      01047        NaN        NaN        NaN        NaN   
          DeKalb      01049        NaN        NaN        NaN        NaN   
          Elmore      01051        NaN        NaN        NaN        NaN   
          Escambia    01053        NaN        NaN        NaN        NaN   
          Etowah      01055        NaN        NaN        NaN        NaN   
          Fayette     01057        NaN        NaN        NaN        NaN   
          Franklin    01059        NaN        NaN        NaN        NaN   
...                                ...        ...        ...        ...   
Wisconsin Sheboygan   55117        NaN        NaN        NaN        NaN   
          St. Croix   55109        NaN        NaN        NaN        NaN   
          Trempealeau 55121        NaN        NaN        NaN        NaN   
          Vilas       55125        NaN        NaN        NaN        NaN   
          Walworth    55127        NaN        NaN        NaN        NaN   
          Washington  55131        NaN        NaN        NaN        NaN   
          Waukesha    55133        NaN        NaN        NaN        NaN   
          Waupaca     55135        NaN        NaN        NaN        NaN   
          Waushara    55137        NaN        NaN        NaN        NaN   
          Winnebago   55139        NaN        NaN        NaN        NaN   
          Wood        55141        NaN        NaN        NaN        NaN   
Wyoming   Albany      56001        NaN        NaN        NaN        NaN   
          Campbell    56005        NaN        NaN        NaN        NaN   
          Carbon      56007        NaN        NaN        NaN        NaN   
          Converse    56009        NaN        NaN        NaN        NaN   
          Crook       56011        NaN        NaN        NaN        NaN   
          Fremont     56013        NaN        NaN        NaN        NaN   
          Goshen      56015        NaN        NaN        NaN        NaN   
          Hot Springs 56017        NaN        NaN        NaN        NaN   
          Johnson     56019        NaN        NaN        N

Add Cases Time Series, First Case Date, Death Time Series, First Death Date

In [4]:
pivot_counties['cases_ts'] = pivot_counties['cases'].values.tolist()
pivot_counties['cases_ts'] = pivot_counties['cases_ts'].apply(lambda x: ','.join(map(str, x)))
pivot_counties['deaths_ts'] = pivot_counties['deaths'].values.tolist()
pivot_counties['deaths_ts'] = pivot_counties['deaths_ts'].apply(lambda x: ','.join(map(str, x)))


In [5]:
pivot_counties['dt_first_case'] = (pivot_counties['cases'] > 0).idxmax(axis=1)
pivot_counties['dt_first_death'] = (pivot_counties['deaths'] > 0).idxmax(axis=1)
pivot_counties.loc[pivot_counties['deaths'].iloc[:, -1] <= 0, 'dt_first_death'] = np.nan
pivot_counties

cases                                   \
date                        2020-01-21 2020-01-22 2020-01-23 2020-01-24   
state     county      fips                                                
Alabama   Autauga     01001        NaN        NaN        NaN        NaN   
          Baldwin     01003        NaN        NaN        NaN        NaN   
          Barbour     01005        NaN        NaN        NaN        NaN   
          Bibb        01007        NaN        NaN        NaN        NaN   
          Blount      01009        NaN        NaN        NaN        NaN   
          Bullock     01011        NaN        NaN        NaN        NaN   
          Butler      01013        NaN        NaN        NaN        NaN   
          Calhoun     01015        NaN        NaN        NaN        NaN   
          Chambers    01017        NaN        NaN        NaN        NaN   
          Cherokee    01019        NaN        NaN        NaN        NaN   
          Chilton     01021        NaN        NaN        NaN        NaN   
          Choctaw     01023        NaN        NaN        NaN        NaN   
          Clarke      01025        NaN        NaN        NaN        NaN   
          Clay        01027        NaN        NaN        NaN        NaN   
          Cleburne    01029        NaN        NaN        NaN        NaN   
          Coffee      01031        NaN        NaN        NaN        NaN   
          Colbert     01033        NaN        NaN        NaN        NaN   
          Conecuh     01035        NaN        NaN        NaN        NaN   
          Coosa       01037        NaN        NaN        NaN        NaN   
          Covington   01039        NaN        NaN        NaN        NaN   
          Crenshaw    01041        NaN        NaN        NaN        NaN   
          Cullman     01043        NaN        NaN        NaN        NaN   
          Dale        01045        NaN        NaN        NaN        NaN   
          Dallas      01047        NaN        NaN        NaN        NaN   
          DeKalb      01049        NaN        NaN        NaN        NaN   
          Elmore      01051        NaN        NaN        NaN        NaN   
          Escambia    01053        NaN        NaN        NaN        NaN   
          Etowah      01055        NaN        NaN        NaN        NaN   
          Fayette     01057        NaN        NaN        NaN        NaN   
          Franklin    01059        NaN        NaN        NaN        NaN   
...                                ...        ...        ...        ...   
Wisconsin Sheboygan   55117        NaN        NaN        NaN        NaN   
          St. Croix   55109        NaN        NaN        NaN        NaN   
          Trempealeau 55121        NaN        NaN        NaN        NaN   
          Vilas       55125        NaN        NaN        NaN        NaN   
          Walworth    55127        NaN        NaN        NaN        NaN   
          Washington  55131        NaN        NaN        NaN        NaN   
          Waukesha    55133        NaN        NaN        NaN        NaN   
          Waupaca     55135        NaN        NaN        NaN        NaN   
          Waushara    55137        NaN        NaN        NaN        NaN   
          Winnebago   55139        NaN        NaN        NaN        NaN   
          Wood        55141        NaN        NaN        NaN        NaN   
Wyoming   Albany      56001        NaN        NaN        NaN        NaN   
          Campbell    56005        NaN        NaN        NaN        NaN   
          Carbon      56007        NaN        NaN        NaN        NaN   
          Converse    56009        NaN        NaN        NaN        NaN   
          Crook       56011        NaN        NaN        NaN        NaN   
          Fremont     56013        NaN        NaN        NaN        NaN   
          Goshen      56015        NaN        NaN        NaN        NaN   
          Hot Springs 56017        NaN        NaN        NaN        NaN   
          Johnson     56019        NaN        NaN        N

In [6]:
report_df = pivot_counties[['cases_ts','deaths_ts','dt_first_case','dt_first_death']]
report_df = report_df.reset_index()

In [7]:
report_df.head(5)

,state,county,fips,cases_ts,deaths_ts,dt_first_case,dt_first_death
date,,,,,,,
0,Alabama,Autauga,01001,"nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,na...","nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,na...",2020-03-24,2020-04-06
1,Alabama,Baldwin,01003,"nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,na...","nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,na...",2020-03-14,2020-04-01
2,Alabama,Barbour,01005,"nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,na...","nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,na...",2020-04-03,NaN
3,Alabama,Bibb,01007,"nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,na...","nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,na...",2020-03-30,NaN
4,Alabama,Blount,01009,"nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,na...","nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,na...",2020-03-25,NaN


In [8]:
report_df.columns = ['state','county','fips','cases_ts','deaths_ts','dt_first_case','dt_first_death']

In [9]:
report_df.head(5)

,state,county,fips,cases_ts,deaths_ts,dt_first_case,dt_first_death
0,Alabama,Autauga,01001,"nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,na...","nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,na...",2020-03-24,2020-04-06
1,Alabama,Baldwin,01003,"nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,na...","nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,na...",2020-03-14,2020-04-01
2,Alabama,Barbour,01005,"nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,na...","nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,na...",2020-04-03,NaN
3,Alabama,Bibb,01007,"nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,na...","nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,na...",2020-03-30,NaN
4,Alabama,Blount,01009,"nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,na...","nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,na...",2020-03-25,NaN


Read County Boundary GeoJSON file

In [10]:
old_counties_geojson_df = gpd.read_file(r"D:\UIUC\COVID_19\wherecovid19_webapp\preprocessing\counties_update_new.geojson")
old_counties_geojson_df.head(5)

,NAME,state_name,STATEFP,COUNTYNS,population,geometry
0,Bladen,North Carolina,37,01026336,33778,"MULTIPOLYGON (((-78.90200 34.83527, -78.79960 ..."
1,Stanly,North Carolina,37,01025844,61114,"MULTIPOLYGON (((-80.49738 35.20210, -80.29542 ..."
2,Summit,Ohio,39,01074088,541810,"MULTIPOLYGON (((-81.68699 41.13596, -81.68495 ..."
3,Sullivan,Pennsylvania,42,01213687,6177,"MULTIPOLYGON (((-76.81373 41.59003, -76.22014 ..."
4,Upshur,Texas,48,01384015,40769,"MULTIPOLYGON (((-95.15274 32.66095, -95.15211 ..."


In [11]:
final_df = pd.merge(old_counties_geojson_df[["NAME", "state_name", "population", "geometry"]], report_df, how='right', left_on=['NAME','state_name'], right_on = ['county','state'])
final_df.columns

Index(['NAME', 'state_name', 'population', 'geometry', 'state', 'county',
       'fips', 'cases_ts', 'deaths_ts', 'dt_first_case', 'dt_first_death'],
      dtype='object')

In [12]:
final_df=final_df[["NAME", "state_name", "population", "fips", "dt_first_case", "dt_first_death", "cases_ts", "deaths_ts", "geometry"]]
final_df

,NAME,state_name,population,fips,dt_first_case,dt_first_death,cases_ts,deaths_ts,geometry
0,Bladen,North Carolina,33778.0,37017,2020-03-29,NaN,"nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,na...","nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,na...","MULTIPOLYGON (((-78.90200 34.83527, -78.79960 ..."
1,Stanly,North Carolina,61114.0,37167,2020-03-20,NaN,"nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,na...","nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,na...","MULTIPOLYGON (((-80.49738 35.20210, -80.29542 ..."
2,Summit,Ohio,541810.0,39153,2020-03-14,2020-03-27,"nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,na...","nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,na...","MULTIPOLYGON (((-81.68699 41.13596, -81.68495 ..."
3,Sullivan,Pennsylvania,6177.0,42113,2020-04-04,NaN,"nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,na...","nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,na...","MULTIPOLYGON (((-76.81373 41.59003, -76.22014 ..."
4,Upshur,Texas,40769.0,48459,2020-03-20,NaN,"nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,na...","nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,na...","MULTIPOLYGON (((-95.15274 32.66095, -95.15211 ..."
5,Brown,Texas,37834.0,48049,2020-03-20,NaN,"nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,na...","nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,na...","MULTIPOLYGON (((-99.19587 32.07923, -99.11866 ..."
6,Cherokee,South Carolina,56711.0,45021,2020-04-02,NaN,"nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,na...","nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,na...","MULTIPOLYGON (((-81.87441 35.18352, -81.76813 ..."
7,Cullman,Alabama,82313.0,01043,2020-03-20,2020-04-01,"nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,na...","nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,na...","MULTIPOLYGON (((-87.11199 33.99238, -87.10991 ..."
8,Kewaunee,Wisconsin,20360.0,55061,2020-04-06,NaN,"nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,na...","nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,na...","MULTIPOLYGON (((-87.76131 44.65370, -87.74841 ..."
9,Fulton,Indiana,20212.0,18049,2020-03-27,NaN,"nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,na...","nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,na...","MULTIPOLYGON (((-86.46705 41.17144, -86.07755 ..."


In [13]:
final_df.to_file(r"./nyt_counties_data.geojson", driver='GeoJSON', encoding='utf-8')
print("done")

done


Generate JSON file

In [14]:
county_list = []
for state in df_counties_original.state.unique():
    for county in df_counties_original.loc[(df_counties_original['state'] == state)].county.unique():
        county_list.append({"name": county, "state": state})
 
print(len(county_list), county_list)

2573 [{'name': 'Snohomish', 'state': 'Washington'}, {'name': 'Spokane', 'state': 'Washington'}, {'name': 'King', 'state': 'Washington'}, {'name': 'Walla Walla', 'state': 'Washington'}, {'name': 'Chelan', 'state': 'Washington'}, {'name': 'Clark', 'state': 'Washington'}, {'name': 'Jefferson', 'state': 'Washington'}, {'name': 'Pierce', 'state': 'Washington'}, {'name': 'Kittitas', 'state': 'Washington'}, {'name': 'Kitsap', 'state': 'Washington'}, {'name': 'Skagit', 'state': 'Washington'}, {'name': 'Whatcom', 'state': 'Washington'}, {'name': 'Island', 'state': 'Washington'}, {'name': 'Thurston', 'state': 'Washington'}, {'name': 'Columbia', 'state': 'Washington'}, {'name': 'Grays Harbor', 'state': 'Washington'}, {'name': 'Yakima', 'state': 'Washington'}, {'name': 'Grant', 'state': 'Washington'}, {'name': 'Klickitat', 'state': 'Washington'}, {'name': 'Lewis', 'state': 'Washington'}, {'name': 'Lincoln', 'state': 'Washington'}, {'name': 'Franklin', 'state': 'Washington'}, {'name': 'Mason', 'sta

In [15]:
date= pivot_counties['cases'].columns
date

Index(['2020-01-21', '2020-01-22', '2020-01-23', '2020-01-24', '2020-01-25',
       '2020-01-26', '2020-01-27', '2020-01-28', '2020-01-29', '2020-01-30',
       '2020-01-31', '2020-02-01', '2020-02-02', '2020-02-03', '2020-02-04',
       '2020-02-05', '2020-02-06', '2020-02-07', '2020-02-08', '2020-02-09',
       '2020-02-10', '2020-02-11', '2020-02-12', '2020-02-13', '2020-02-14',
       '2020-02-15', '2020-02-16', '2020-02-17', '2020-02-18', '2020-02-19',
       '2020-02-20', '2020-02-21', '2020-02-22', '2020-02-23', '2020-02-24',
       '2020-02-25', '2020-02-26', '2020-02-27', '2020-02-28', '2020-02-29',
       '2020-03-01', '2020-03-02', '2020-03-03', '2020-03-04', '2020-03-05',
       '2020-03-06', '2020-03-07', '2020-03-08', '2020-03-09', '2020-03-10',
       '2020-03-11', '2020-03-12', '2020-03-13', '2020-03-14', '2020-03-15',
       '2020-03-16', '2020-03-17', '2020-03-18', '2020-03-19', '2020-03-20',
       '2020-03-21', '2020-03-22', '2020-03-23', '2020-03-24', '2020-03-25',

In [16]:
date_str = np.sort(date)
dt_str_start=np.min(date_str)
dt_str_end=np.max(date_str)

In [17]:
dt_start = datetime.strptime(dt_str_start, "%Y-%m-%d")
dt_end = datetime.strptime(dt_str_end, "%Y-%m-%d")
dt_range = pd.date_range(start=dt_start,end=dt_end)
print(len(dt_range), dt_range)
dt_range_str = list(map(lambda x: x.strftime("%Y-%m-%d"), dt_range.tolist()))
print(len(dt_range_str), dt_range_str)

78 DatetimeIndex(['2020-01-21', '2020-01-22', '2020-01-23', '2020-01-24',
               '2020-01-25', '2020-01-26', '2020-01-27', '2020-01-28',
               '2020-01-29', '2020-01-30', '2020-01-31', '2020-02-01',
               '2020-02-02', '2020-02-03', '2020-02-04', '2020-02-05',
               '2020-02-06', '2020-02-07', '2020-02-08', '2020-02-09',
               '2020-02-10', '2020-02-11', '2020-02-12', '2020-02-13',
               '2020-02-14', '2020-02-15', '2020-02-16', '2020-02-17',
               '2020-02-18', '2020-02-19', '2020-02-20', '2020-02-21',
               '2020-02-22', '2020-02-23', '2020-02-24', '2020-02-25',
               '2020-02-26', '2020-02-27', '2020-02-28', '2020-02-29',
               '2020-03-01', '2020-03-02', '2020-03-03', '2020-03-04',
               '2020-03-05', '2020-03-06', '2020-03-07', '2020-03-08',
               '2020-03-09', '2020-03-10', '2020-03-11', '2020-03-12',
               '2020-03-13', '2020-03-14', '2020-03-15', '2020-03-16',
   

In [18]:
data_df = report_df.set_index(report_df["county"]+", "+report_df["state"])
data_dict =data_df.to_dict(orient='index')

In [19]:
df = df_counties_original.groupby(['county', 'state'], sort=False)[['cases', 'deaths']].max().sort_values("cases", ascending=False)
county_state_list = list(map(lambda x: x[0]+", "+x[1], df.index.tolist()))
meta = {"dates": dt_range_str,"states": county_state_list, "cases": df["cases"].values.tolist(), "deaths": df["deaths"].values.tolist()}
data_dict["metadata"] = meta

In [20]:
data_dict

{'Autauga, Alabama': {'state': 'Alabama',
  'county': 'Autauga',
  'fips': '01001',
  'cases_ts': 'nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,1.0,4.0,6.0,6.0,6.0,6.0,7.0,7.0,10.0,10.0,12.0,12.0,12.0,12.0,12.0',
  'deaths_ts': 'nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0',
  'dt_first_case': '2020-03-24',
  'dt_first_death': '2020-04-06'},
 'Baldwin, Alabama': {'state': 'Alabama',
  'county': 'Baldwin',
  'fips': '01003',
  'cases_ts': 'nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan

In [21]:
import json
with open('./nyt_counties_data.json', 'w') as outfile:
    json.dump(data_dict, outfile)